In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/My Drive/Capstone") 

In [4]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [5]:
df = pd.read_csv('./portfolio_13F_new.csv')

In [6]:
len(df['INSTRID'].value_counts())

11013

In [7]:
df.head()

,INSTRID,COMNAME,LGCYINVESTORID,FULLNAME,HOLDDATE,SHSHLDVAL,SHSHLDVAL_Sum_ByQuarter,PCTPORTFOLIO_13F
0,8589934603,First Trust Consumer Discretionary AlphaDEX ETF,2003020,BofA Global Research (US),2020-06-30 00:00:00,15119117.23,3.439259e+11,0.000044
1,8589934603,First Trust Consumer Discretionary AlphaDEX ETF,2003020,BofA Global Research (US),2017-06-30 00:00:00,12815061.00,2.976255e+11,0.000043
2,8589934603,First Trust Consumer Discretionary AlphaDEX ETF,2003020,BofA Global Research (US),2017-12-31 00:00:00,14477360.58,3.230778e+11,0.000045
3,8589934603,First Trust Consumer Discretionary AlphaDEX ETF,2003020,BofA Global Research (US),2018-03-31 00:00:00,12468032.28,3.066132e+11,0.000041
4,8589934603,First Trust Consumer Discretionary AlphaDEX ETF,2003020,BofA Global Research (US),2017-03-31 00:00:00,16205549.78,2.879759e+11,0.000056


In [8]:
mktcap = pd.read_csv('./MktCap_MktCapSize_20160630_20200630.csv')
mktcap.head()

,InstrId,EffectDate,PrcCloseAct,NumOfShOut,MktCap,MktCapSize
0,8589934595,2016-06-30,0.02,29520523,5.904105e+05,NanoCap
1,8589934597,2016-06-30,18.51,16347483,3.025919e+08,SmallCap
2,8589934599,2016-06-30,0.07,40000000,2.800000e+06,NanoCap
3,8589934600,2016-06-30,1.21,2750000,3.327500e+06,NanoCap
4,8589934603,2016-06-30,33.72,50700002,1.709604e+09,SmallCap


In [9]:
len(mktcap['InstrId'].value_counts())

123113

In [10]:
df['LGCYINVESTORID'] = df['LGCYINVESTORID'].astype(str)
df['INSTRID'] = df['INSTRID'].astype(str)
df['HOLDDATE'] = df['HOLDDATE'].str.replace(r' 00:00:00', '')
# df['LGCYINVESTORID_HOLDDATE'] =  df_sub[['LGCYINVESTORID', 'HOLDDATE']].apply(lambda x: '_'.join(x), axis=1)

In [11]:
df.head()

,INSTRID,COMNAME,LGCYINVESTORID,FULLNAME,HOLDDATE,SHSHLDVAL,SHSHLDVAL_Sum_ByQuarter,PCTPORTFOLIO_13F
0,8589934603,First Trust Consumer Discretionary AlphaDEX ETF,2003020,BofA Global Research (US),2020-06-30,15119117.23,3.439259e+11,0.000044
1,8589934603,First Trust Consumer Discretionary AlphaDEX ETF,2003020,BofA Global Research (US),2017-06-30,12815061.00,2.976255e+11,0.000043
2,8589934603,First Trust Consumer Discretionary AlphaDEX ETF,2003020,BofA Global Research (US),2017-12-31,14477360.58,3.230778e+11,0.000045
3,8589934603,First Trust Consumer Discretionary AlphaDEX ETF,2003020,BofA Global Research (US),2018-03-31,12468032.28,3.066132e+11,0.000041
4,8589934603,First Trust Consumer Discretionary AlphaDEX ETF,2003020,BofA Global Research (US),2017-03-31,16205549.78,2.879759e+11,0.000056


In [12]:
len(df['LGCYINVESTORID'].value_counts())

229

In [13]:
active_list = pd.read_csv('./active_investor_list.csv')
active_list['LgcyInvestorId'] = active_list['LgcyInvestorId'].astype(str)

In [14]:
active_list.head()

,LgcyInvestorId,ActivePct,RecordCount,isActive,FULLNAME
0,2000264,1.000000,23,True,"Lone Pine Capital, L.L.C."
1,2001934,0.693642,346,True,Voya Investment Management LLC
2,2001935,1.000000,6,True,"UBS Financial Services, Inc."
3,2001989,0.904762,21,True,"Ariel Investments, LLC"
4,2001999,0.890733,723,True,JP Morgan Asset Management


In [15]:
active_inv = active_list[active_list['isActive'] == True]['LgcyInvestorId'].values.tolist()
len(active_inv)

184

In [17]:
active_df = df[df['LGCYINVESTORID'].isin(active_inv)]
active_df

,INSTRID,COMNAME,LGCYINVESTORID,FULLNAME,HOLDDATE,SHSHLDVAL,SHSHLDVAL_Sum_ByQuarter,PCTPORTFOLIO_13F
38,8589934707,Hill International Ord Shs,2578454,"Two Sigma Investments, LP",2016-09-30,446330.98,4.708608e+10,9.479043e-06
40,8589934707,Hill International Ord Shs,2578454,"Two Sigma Investments, LP",2016-12-31,260038.65,4.931067e+10,5.273476e-06
60,8589934707,Hill International Ord Shs,2003003,"BlackRock Investment Management, LLC",2016-09-30,672788.01,8.125913e+10,8.279538e-06
61,8589934707,Hill International Ord Shs,2003003,"BlackRock Investment Management, LLC",2016-06-30,596832.94,7.890116e+10,7.564311e-06
62,8589934707,Hill International Ord Shs,2003003,"BlackRock Investment Management, LLC",2020-06-30,16703.28,1.117799e+11,1.494301e-07
...,...,...,...,...,...,...,...,...
3737805,25769769830,Brookfield Business Partners Units,2002713,"U.S. Bancorp Asset Management, Inc.",2019-03-31,2857.01,3.463121e+10,8.249814e-08
3737806,25769769830,Brookfield Business Partners Units,2002713,"U.S. Bancorp Asset Management, Inc.",2017-12-31,2478.24,3.268584e+10,7.581999e-08
3737807,25769769830,Brookfield Business Partners Units,2002713,"U.S. Bancorp Asset Management, Inc.",2017-06-30,2624.43,2.994941e+10,8.762876e-08
3737808,25769769830,Brookfield Business Partners Units,2002713,"U.S. Bancorp Asset Management, Inc.",2018-03-31,2595.62,3.256537e+10,7.970491e-08


In [18]:
mktcap['InstrId'] = mktcap['InstrId'].astype(str)

In [19]:
merge = pd.merge(active_df, mktcap, how='inner', left_on=['INSTRID', 'HOLDDATE'], right_on=['InstrId', 'EffectDate'])

In [20]:
merge.drop(columns=['InstrId', 'EffectDate'])

,INSTRID,COMNAME,LGCYINVESTORID,FULLNAME,HOLDDATE,SHSHLDVAL,SHSHLDVAL_Sum_ByQuarter,PCTPORTFOLIO_13F,PrcCloseAct,NumOfShOut,MktCap,MktCapSize
0,8589934707,Hill International Ord Shs,2578454,"Two Sigma Investments, LP",2016-09-30,446330.98,4.708608e+10,9.479043e-06,4.6100,51748900,2.385624e+08,MicroCap
1,8589934707,Hill International Ord Shs,2003003,"BlackRock Investment Management, LLC",2016-09-30,672788.01,8.125913e+10,8.279538e-06,4.6100,51748900,2.385624e+08,MicroCap
2,8589934707,Hill International Ord Shs,2002468,"Northern Trust Investments, Inc.",2016-09-30,1535346.67,1.993541e+11,7.701607e-06,4.6100,51748900,2.385624e+08,MicroCap
3,8589934707,Hill International Ord Shs,2002658,"Dimensional Fund Advisors, L.P.",2016-09-30,9591602.88,1.849165e+11,5.186991e-05,4.6100,51748900,2.385624e+08,MicroCap
4,8589934707,Hill International Ord Shs,2003171,AXA Rosenberg Investment Management LLC,2016-09-30,741490.84,1.320253e+10,5.616276e-05,4.6100,51748900,2.385624e+08,MicroCap
...,...,...,...,...,...,...,...,...,...,...,...,...
2370081,25769769830,Brookfield Business Partners Units,2003332,Principal Global Investors (Equity),2020-06-30,16655037.65,1.135159e+11,1.467199e-04,30.7891,80890655,2.490550e+09,MidCap
2370082,25769769830,Brookfield Business Partners Units,2002389,Paloma Partners Management Company,2020-06-30,223374.68,4.704608e+09,4.747998e-05,30.7891,80890655,2.490550e+09,MidCap
2370083,25769769830,Brookfield Business Partners Units,2002713,"U.S. Bancorp Asset Management, Inc.",2020-06-30,2216.81,3.809195e+10,5.819629e-08,30.7891,80890655,2.490550e+09,MidCap
2370084,44645091900,Entergy Arkansas First Mortgage Bonds 4.90% Se...,2002848,Columbia Threadneedle Investments (US),2016-12-31,251356.42,1.665482e+11,1.509211e-06,21.7700,8000000,1.741600e+08,MicroCap


In [21]:
len(merge['INSTRID'].value_counts())

10931

In [92]:
agg_mktcap_pivot = pd.pivot_table(merge,index = ['LGCYINVESTORID', 'HOLDDATE'], columns = ['MktCapSize'],values=['PCTPORTFOLIO_13F'],aggfunc=[np.sum],fill_value=0)
# inves_instr_pivot.columns = inves_instr_pivot.columns.droplevel()

In [93]:
agg_mktcap_pivot

sum            ...                        
                          PCTPORTFOLIO_13F            ...                        
MktCapSize                        LargeCap   MegaCap  ...       NanoCap  SmallCap
LGCYINVESTORID HOLDDATE                               ...                        
10075382       2018-12-31         0.786460  0.085594  ...  0.000000e+00  0.000041
               2019-03-31         0.776636  0.111391  ...  0.000000e+00  0.000000
               2019-06-30         0.768383  0.180804  ...  0.000000e+00  0.000000
               2019-09-30         0.723701  0.211930  ...  0.000000e+00  0.000000
               2019-12-31         0.728066  0.231642  ...  0.000000e+00  0.001348
...                                    ...       ...  ...           ...       ...
9113038        2019-06-30         0.481773  0.304416  ...  0.000000e+00  0.044197
               2019-09-30         0.486167  0.302835  ...  0.000000e+00  0.047257
               2019-12-31         0.477020  0.312607  ...  0.000000e+00  0.042244
               2020-03-31         0.494611  0.283755  ...  1.581639e-05  0.060636
               2020-06-30         0.472083  0.316396  ...  1.421072e-07  0.051942

[2996 rows x 6 columns]

In [94]:
investor_set = set(agg_mktcap_pivot.index.get_level_values('LGCYINVESTORID'))
print(len(investor_set))

184


In [95]:
holddate_set = set(agg_mktcap_pivot.index.get_level_values('HOLDDATE'))
print(len(holddate_set))

17


In [96]:
print(len(investor_set)*len(holddate_set))
print(len(agg_mktcap_pivot))

3128
2996


In [97]:
agg_mktcap_pivot[agg_mktcap_pivot.index.get_level_values('LGCYINVESTORID') == '10075382'].index.get_level_values('HOLDDATE').to_list()

['2018-12-31',
 '2019-03-31',
 '2019-06-30',
 '2019-09-30',
 '2019-12-31',
 '2020-03-31',
 '2020-06-30']

In [98]:
mktcap_cat = ['PctLargeCap', 'MegaCap', 'MicroCap', 'MidCap', 'NanoCap', 'SmallCap']

In [99]:
agg_mktcap_pivot.columns = mktcap_cat

In [100]:
agg_mktcap_pivot = agg_mktcap_pivot.reset_index()

In [101]:
for inv in investor_set:
  holddates = set(agg_mktcap_pivot[agg_mktcap_pivot['LGCYINVESTORID']==inv]['HOLDDATE'].to_list())
  if len(holddates) < len(holddate_set):
    print('Investor ID:', inv, ' missing:')
    print(holddate_set - holddates)
    missing_dates = holddate_set - holddates
    for md in missing_dates:
      agg_mktcap_pivot = agg_mktcap_pivot.append([{'LGCYINVESTORID': inv, 'HOLDDATE':md}])

Investor ID: 2930530  missing:
{'2017-12-31'}
Investor ID: 5206490  missing:
{'2020-03-31', '2020-06-30'}
Investor ID: 5308615  missing:
{'2016-12-31', '2016-09-30', '2017-09-30', '2017-06-30', '2017-03-31'}
Investor ID: 2003597  missing:
{'2018-09-30', '2016-06-30', '2016-12-31', '2018-06-30', '2016-09-30', '2017-09-30', '2017-06-30', '2017-12-31', '2018-03-31', '2017-03-31'}
Investor ID: 2502059  missing:
{'2020-03-31'}
Investor ID: 2646754  missing:
{'2018-09-30', '2016-12-31', '2018-06-30', '2016-09-30', '2017-09-30', '2017-06-30', '2017-12-31', '2019-09-30', '2017-03-31', '2019-06-30', '2019-03-31', '2018-12-31', '2018-03-31', '2016-06-30'}
Investor ID: 3965008  missing:
{'2020-03-31', '2020-06-30', '2016-06-30'}
Investor ID: 5200477  missing:
{'2018-09-30', '2016-06-30', '2016-12-31', '2018-06-30', '2019-12-31', '2016-09-30', '2017-06-30', '2017-09-30', '2017-12-31', '2018-03-31', '2020-03-31', '2020-06-30', '2017-03-31'}
Investor ID: 4962258  missing:
{'2018-09-30', '2016-12-31'

In [102]:
agg_mktcap_pivot = agg_mktcap_pivot.fillna(0)

In [104]:
agg_mktcap_pivot.head()

,LGCYINVESTORID,HOLDDATE,PctLargeCap,MegaCap,MicroCap,MidCap,NanoCap,SmallCap
0,10075382,2018-12-31,0.786460,0.085594,0.0,0.127904,0.0,0.000041
1,10075382,2019-03-31,0.776636,0.111391,0.0,0.111974,0.0,0.000000
2,10075382,2019-06-30,0.768383,0.180804,0.0,0.050812,0.0,0.000000
3,10075382,2019-09-30,0.723701,0.211930,0.0,0.064368,0.0,0.000000
4,10075382,2019-12-31,0.728066,0.231642,0.0,0.038943,0.0,0.001348


In [ ]:
agg_mktcap_pivot.to_csv('./aggregated_mktcap_pct.csv', index=False)

In [110]:
def cal_distance(invID_1, invID_2, agg_mktcap_pivot = agg_mktcap_pivot):
  distance = []
  for cat in mktcap_cat:
    ts_inv_1 = agg_mktcap_pivot[agg_mktcap_pivot['LGCYINVESTORID'] == invID_1][cat].values
    ts_inv_2 = agg_mktcap_pivot[agg_mktcap_pivot['LGCYINVESTORID'] == invID_2][cat].values
    distance.append(np.linalg.norm(ts_inv_1-ts_inv_2))
  return np.mean(distance)

In [106]:
investor_list = list(set(merge['LGCYINVESTORID']))
# len(investor_list)

In [111]:
distance_matrix = np.zeros((len(investor_list), len(investor_list)))
for i in tqdm(range(len(investor_list))):
  for j in range(i+1, len(investor_list)):
    distance_matrix[i][j] = cal_distance(investor_list[i], investor_list[j])
    distance_matrix[j][i] = distance_matrix[i][j]

In [112]:
distance_matrix

array([[0.        , 0.50909254, 1.24277933, ..., 0.50230091, 0.24039319,
        0.16793423],
       [0.50909254, 0.        , 0.99643481, ..., 0.46847486, 0.55268867,
        0.40085093],
       [1.24277933, 0.99643481, 0.        , ..., 1.00854499, 1.22381757,
        1.10690499],
       ...,
       [0.50230091, 0.46847486, 1.00854499, ..., 0.        , 0.52825776,
        0.45372385],
       [0.24039319, 0.55268867, 1.22381757, ..., 0.52825776, 0.        ,
        0.29955141],
       [0.16793423, 0.40085093, 1.10690499, ..., 0.45372385, 0.29955141,
        0.        ]])

In [113]:
distance_matrix_df = pd.DataFrame(distance_matrix)

In [116]:
distance_matrix_df.head()

,2003710,5206490,2930530,2004490,2006430,2002142,4924814,2001999,2002947,2004627,2003800,2001935,5308615,2452904,2003332,2003597,2529162,2002158,5206923,2004282,2412347,2002875,2502059,5050280,2625886,2002628,2002921,5529187,2002975,2004887,2449877,2004134,2575181,2005893,2455426,3015075,2002819,2646754,2001934,2003819,...,2113769,5986931,2007217,2785553,2002713,2006033,2004523,2007403,2536411,2004318,7016524,2003003,2002451,2003838,2620417,2003101,2003171,2006588,2002180,2005821,5205742,2006922,2003294,2578454,2003230,2490230,2002867,2002801,2906273,2005801,2006955,2007683,2007039,2004000,2247678,6921752,6139212,9113038,4040011,2003354
2003710,0.000000,0.509093,1.242779,0.394200,1.040254,0.158446,1.161986,0.172539,0.446653,0.185420,0.054327,0.216783,0.876947,0.549795,0.330080,0.564718,0.480378,0.206758,0.360280,0.533095,0.552986,0.172986,0.848882,0.273353,0.821388,0.148086,0.149879,0.253527,0.189398,0.508881,0.288534,0.245198,0.379033,0.514540,0.193307,1.022811,0.159554,0.657331,0.250448,0.050423,...,0.241158,1.246076,0.571354,0.214605,0.087683,0.210751,1.189464,0.448782,0.681872,1.410602,1.299881,0.167352,0.215302,0.394419,0.927089,0.516881,0.126878,0.932344,0.485838,0.318736,0.921086,0.354014,0.311166,0.380324,0.572769,1.018972,0.235661,0.152434,1.365069,0.611091,0.838101,0.648030,0.193986,0.054906,0.214409,0.667783,0.411153,0.502301,0.240393,0.167934
5206490,0.509093,0.000000,0.996435,0.366505,0.783352,0.450786,0.861755,0.400034,0.413662,0.484749,0.478067,0.372851,0.789567,0.404148,0.383423,0.520471,0.483324,0.456852,0.473015,0.454253,0.404043,0.404286,0.755571,0.436967,0.668733,0.491060,0.431365,0.411977,0.471344,0.406680,0.418242,0.389955,0.399636,0.436081,0.401441,0.757787,0.440383,0.704498,0.398503,0.480636,...,0.416060,0.913156,0.398457,0.466988,0.461462,0.485834,0.918443,0.362751,0.670895,1.117937,1.108371,0.410708,0.405497,0.409247,0.733606,0.391289,0.419085,0.629870,0.417767,0.430085,0.683899,0.389521,0.353462,0.373228,0.444179,0.711317,0.384491,0.437568,1.069408,0.536206,0.552513,0.573427,0.419040,0.478099,0.499075,0.586739,0.467229,0.468475,0.552689,0.400851
2930530,1.242779,0.996435,0.000000,0.929828,1.036060,1.163532,1.029400,1.103564,0.875322,1.212612,1.208295,1.088234,0.986769,0.832996,0.970565,0.988439,1.072718,1.134756,1.099547,0.896505,0.846339,1.098886,1.160034,1.087809,0.694752,1.237509,1.168060,1.133216,1.209118,1.008422,1.051478,1.043613,0.925807,0.859167,1.096764,1.015117,1.156381,1.049397,1.041454,1.212534,...,1.039935,1.099346,0.881449,1.142394,1.192563,1.222466,0.725942,0.987425,0.928708,1.308642,1.399303,1.117285,1.067662,0.971727,0.672509,0.934188,1.145616,0.871806,0.969776,1.016416,0.797374,0.949228,1.026481,0.947135,0.790653,0.654773,1.042316,1.132134,1.107297,0.845822,0.763643,0.974999,1.089561,1.207560,1.151542,0.933921,1.100486,1.008545,1.223818,1.106905
2004490,0.394200,0.366505,0.929828,0.000000,0.771061,0.321386,0.832681,0.234451,0.127784,0.386751,0.355443,0.201179,0.733013,0.248178,0.091458,0.571939,0.360523,0.329471,0.394091,0.265780,0.177544,0.231601,0.743741,0.266314,0.528293,0.384372,0.303579,0.272683,0.364385,0.306915,0.217848,0.166272,0.085418,0.182669,0.219737,0.734598,0.311220,0.720684,0.172290,0.360721,...,0.191980,0.957764,0.199389,0.314599,0.336290,0.385129,0.839576,0.205393,0.693063,1.169129,1.194371,0.238952,0.203081,0.176441,0.628346,0.274941,0.280819,0.587986,0.231381,0.215515,0.594894,0.160256,0.134029,0.109819,0.234349,0.659674,0.171316,0.289349,1.046234,0.389953,0.492395,0.532750,0.222896,0.354178,0.320553,0.390189,0.506814,0.485797,0.446376,0.238950
2006430,1.040254,0.783352,1.036060,0.771061,0.000000,0.975406,0.760786,0.913941,0.788067,1.024682,1.008480,0.876381,0.938746,0.735841,0.817005,0.978588,0.813049,0.963754,0.901405,0.721198,0.707992,0.922035,1.066405,0.941647,0.705987,1.011175,0.948898,0.901772,0.991030,0.811086,0.887908,0.883208,0.811289,0.753020,0.908758,0.626327,0.948985,1.049034,0.880022,1.008241,...,0.884162,0.712099,0.649694,0.999537,0.988391,1.002775,0

In [115]:
distance_matrix_df.columns = investor_list
distance_matrix_df.index = investor_list

In [117]:
distance_matrix_df.to_csv('./distance_matrix_df_by_pct_mktcap.csv')

In [ ]:
############################################### 

In [ ]:
vanguard_id = '2004260'
investor_list = list(set(merge['LGCYINVESTORID']))

In [ ]:
cal_distance(vanguard_id, '10061989')

0.4254830753347956

In [ ]:
distance_to_vanguard_list = []
for investor in tqdm(investor_list):
  distance_to_vanguard = cal_distance(investor, vanguard_id)
  distance_to_vanguard_list.append(distance_to_vanguard)

distance_to_vanguard_df = pd.DataFrame({
    'investor_id': investor_list,
    'distance_to_vanguard': distance_to_vanguard_list
})

In [ ]:
distance_to_vanguard_df = distance_to_vanguard_df.merge(merge[['LGCYINVESTORID','FULLNAME']], how='inner', left_on='investor_id', right_on='LGCYINVESTORID')

In [ ]:
distance_to_vanguard_df = distance_to_vanguard_df.drop_duplicates(subset='investor_id').drop(columns='investor_id')

In [ ]:
distance_to_vanguard_df = distance_to_vanguard_df.sort_values(by='distance_to_vanguard')

In [ ]:
distance_to_vanguard_df

,distance_to_vanguard,LGCYINVESTORID,FULLNAME
1981677,0.000000,2004260,"The Vanguard Group, Inc."
1461563,0.027857,2002506,BNY Mellon Asset Management
707576,0.029248,2003131,"T. Rowe Price Associates, Inc."
1576774,0.029891,2003003,"BlackRock Investment Management, LLC"
1255342,0.032800,2455426,"Adage Capital Management, L.P."
...,...,...,...
1645962,1.121646,4889409,"Centerbridge Partners, L.P."
2702089,1.210031,2906273,"Great Point Partners, LLC"
1055623,1.230583,7016524,"Hartford Funds Management Company, LLC"
454805,1.271799,2004318,CarVal Investors L.P.


In [ ]:
distance_to_vanguard_df.to_csv('./distance_to_vanguard_by_aggregating_mktcap.csv', index=False)